In [2]:
import pandas as pd
df = pd.read_csv("uploads/a.csv")
df


,ShipmentId,ReverseShipmentId,ForwardReverseType,TotalPrice,DeliveryPincode,Status,DeliveryHub,CurrentHub,CustomerPromiseDate,LogisticsPromiseDate,...,NumberOfAttempts,CsNotes,HubNotes,OnHoldByOpsReason,OnHoldByOpsDate,BagId,BagStatus,ConsignmentId,OrderId,ItemDescription
0,FMPC3687331622,NaN,NORMAL_FORWARD,228.0,751030,Undelivered_Not_Attended,KhandagiriHub_BBI,MotherHub_YKB,2024-05-11 23:59:59,2024-05-11 8:30:00,...,0,NaN,NaN,NaN,NaN,PH-B5-DELYKB-35491357,TERMINATED,161067662.0,OD331181237004604100,RUDRAELEVEN Cotton Arm Sleeve For Men & Women.
1,FMPC3686931339,NaN,NORMAL_FORWARD,139.0,846001,Undelivered_Not_Attended,CABTDarbhangaSplitODH_DBG,MotherHub_YKB,2024-05-10 23:59:59,2024-05-10 11:30:00,...,0,NaN,NaN,NaN,NaN,PH-ZO-DELYKB-35547731,TERMINATED,161050122.0,OD331181390056556400,ShopGlobal Two Wheeler Cover for Honda.
2,FMPC3681505518,NaN,NORMAL_FORWARD,204.0,440027,Undelivered_Not_Attended,WardhamanNagar_NAG,WardhamanNagar_NAG,2024-05-08 23:59:59,2024-05-08 11:30:00,...,0,NaN,NaN,NaN,NaN,PH-ZO-DELYKB-35482941,TERMINATED,160695578.0,OD331164749158856100,Fashionury Back Cover for Realme 11x 5G.
3,MYEP1019951628,NaN,NORMAL_FORWARD,1671.0,400612,Undelivered_Not_Attended,KoorieeMumbraODH_MBR,MotherHub_YKB,2024-05-09 5:30:00,2024-05-10 11:30:00,...,0,NaN,NaN,NaN,NaN,PH-ZO-DELYKB-35493894,TERMINATED,161057146.0,7895650706,SWISS MILITARY Chopmate White Electric Chopper...
4,FMPC3684464564,NaN,NORMAL_FORWARD,474.0,700089,Error,LakeTownHub_KOL,ElasticRunNarodaODH_NRD,2024-05-09 23:59:59,2024-05-09 11:30:00,...,0,NaN,Misrouted to hub : ElasticRunNarodaODH_NRD,NaN,NaN,PH-B5-DELYKB-35483064,TERMINATED,160892458.0,OD431175669427380100,POPWINGS Regular Fit Women Multicolor Trousers.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,OIPC0018143681,NaN,NORMAL_FORWARD,1308.0,760010,Undelivered_Not_Attended,CABTBerhampurODH_BMP,MotherHub_YKB,2024-05-10 18:00:00,2024-05-10 18:00:00,...,0,NaN,NaN,NaN,NaN,PH-B5-DELYKB-35482098,TERMINATED,161093281.0,8490747283SAX,Plane Matty Solid Half Sleeves Mens Polo neck ...
372,FMPC3662031785,NaN,NORMAL_FORWARD,225.0,483113,Undelivered_UntraceableFromHub,IdentifyplusMaharajpurODH_MHR,ElasticRunJaripatkaODH_NAG,2024-05-01 23:59:59,2024-04-30 11:30:00,...,0,UNTRACEABLE_FROM_HUB,Marked_as_Untraceable_from_hub,NaN,NaN,PH-ZO-DELYKB-35076739,TERMINATED,159439528.0,OD331100035347749100,Hilary Rhoda HR-R23-05 ~ Double Wear Cover Pan...
373,SDLC1030569964,NaN,NORMAL_FORWARD,158.0,388540,Undelivered_Not_Attended,AnandHub_AMD,MotherHub_YKB,2024-05-04 18:00:00,2024-05-04 18:00:00,...,0,NaN,NaN,NaN,NaN,PH-B5-DELYKB-35348337,TERMINATED,160140249.0,SLP4200915964,Dr. Morepen BG-03 Glucometer.
374,FMPC3683787347,NaN,NORMAL_FORWARD,260.0,743286,Undelivered_Not_Attended,BasirhatHub_BSH,MotherHub_YKB,2024-05-09 23:59:59,2024-05-09 11:30:00,...,0,NaN,NaN,NaN,NaN,PH-B5-DELYKB-35366274,OPENED,160876601.0,OD431174150081485100,MOBISINGH MSH22CS086TB 22MM Classy Silicon Wat...


In [14]:
df["Remarks"] = ""
df["Bucket"] = ""

df.loc[df['CurrentHub'] == 'MotherHub_YKB', 'Remarks'] = 'Pending at YKB'
df.loc[df['CurrentHub'] == 'MotherHub_YKB', 'Bucket'] = 'YKB'

df.loc[df["CurrentHub"] == df["DeliveryHub"], "Remarks"] = "Delivered at Correct DH/ODH"
df.loc[df["CurrentHub"] == "MotherHub_YKB", "Bucket"] = "Cleared"


df.loc[(df["CurrentHub"] != "MotherHub_YKB") & (df["CurrentHub"] != df["DeliveryHub"]) , "Remarks"] = "Recieced at Wrong DH/ODH"
df.loc[(df["CurrentHub"] != "MotherHub_YKB") & (df["CurrentHub"] != df["DeliveryHub"]) & (~df["Status"].str.contains("Error")), "Remarks"] = "Recieved in Error at Wrong DH/ODH"
df.loc[(df["CurrentHub"] != "MotherHub_YKB") & (df["CurrentHub"] != df["DeliveryHub"]) , "Bucket"] = "Cleared/PWD"
df.loc[(df["CurrentHub"] != "MotherHub_YKB") & (df["CurrentHub"] != df["DeliveryHub"]) & (~df["Status"].str.contains("Error")), "Bucket"] = "Cleared/PWD"

# Create a pivot table
temp_table = df.pivot_table(index="Remarks", columns="Bucket", values="ShipmentId", aggfunc="count", margins=True).fillna(0)

# Show the pivot table
temp_table

Bucket,,Cleared,Cleared/PWD,All
Remarks,,,,
Delivered at Correct DH/ODH,30.0,0.0,0.0,30
Pending at YKB,0.0,249.0,0.0,249
Recieced at Wrong DH/ODH,0.0,0.0,49.0,49
Recieved in Error at Wrong DH/ODH,0.0,0.0,48.0,48
All,30.0,249.0,97.0,376
